In [1]:
# PyTorch 라이브러리 import
import torch
import torch.nn as nn
from torchvision import transforms, datasets

In [2]:
# TODO: CIFAR-100 training set 불러오기
# import random

# random.seed(2024)
# torch.manual_seed(2024)
# torch.cuda.manual_seed_all(2024)
# torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.benchmark = False

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.05),
    transforms.ToTensor(),
    #transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])


train_dataset = datasets.CIFAR10(root='.', train=True, download=True,
                                 transform=transform_train)
# train_size = int(0.8 * len(train_dataset))
# validation_size = len(train_dataset) - train_size
# train_dataset, validation_dataset = torch.utils.data.random_split(train_dataset, [train_size, validation_size])

batch_size = 64
train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True,
                                           num_workers=2,
                                           generator=torch.Generator().manual_seed(2024))
# validation_loader = torch.utils.data.DataLoader(validation_dataset,
#                                                batch_size=batch_size, shuffle=False, num_workers=2)

# classes = ('plane', 'car', 'bird', 'cat', 'deer',
#            'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified


In [8]:
class MLP(nn.Module):
  def __init__(self):
    super(MLP, self).__init__()
    torch.manual_seed(2024) # 결과 재현을 위한 seed number 고정
    # TODO: MLP 구성 layer들 선언
    self.flatten = nn.Flatten()
    self.fc1 = nn.Linear(3*32*32, 2048)
    self.bn1 = nn.BatchNorm1d(2048)
    self.fc2 = nn.Linear(2048, 512)
    self.bn2 = nn.BatchNorm1d(512)
    self.fc3 = nn.Linear(512, 10)
    nn.init.xavier_uniform_(self.fc1.weight)
    nn.init.xavier_uniform_(self.fc2.weight)
    nn.init.xavier_uniform_(self.fc3.weight)
    self.relu = nn.ReLU()
    #self.dropout = nn.Dropout(0.3)

  def forward(self, x):
    x = x.view(-1, 32*32*3) # MLP가 이미지를 처리할 수 있도록 3차원 -> 1차원 벡터로 Flatten
    # TODO: forward pass 정의
    x = self.relu(self.bn1(self.fc1(x)))
    #x = self.dropout(x)
    x = self.relu(self.bn2(self.fc2(x)))
    #x = self.dropout(x)
    x = self.fc3(x)
    return x


model = MLP().cuda()

num_epochs = 50
learning_rate = 0.001
criterion = nn.CrossEntropyLoss()
#optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
#optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=5e-4)
#optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=5e-4, nesterov=True)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=5e-4)
#scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max = num_epochs)

In [9]:
# TODO: model 명의로 생성된 MLP 모델에 대해 학습 수행하기
for epoch in range(num_epochs):
    total = 0
    correct = 0
    train_loss = 0.0

    # training
    model.train()
    print(f'        Epoch [{epoch+1}/{num_epochs}]')
    for batch_idx, (images, labels) in enumerate(train_loader):
        images, labels = images.to('cuda'), labels.to('cuda')
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    train_loss /= len(train_loader)
    train_accuracy = 100. * correct / total

#     # validation
#     model.eval()
#     val_loss = 0.0
#     val_total = 0
#     val_correct = 0
#     with torch.no_grad():
#         for images, labels in validation_loader:
#             images, labels = images.to('cuda'), labels.to('cuda')
#             outputs = model(images)
#             loss = criterion(outputs, labels)

#             val_loss += loss.item()
#             _, predicted = outputs.max(1)
#             val_total += labels.size(0)
#             val_correct += predicted.eq(labels).sum().item()

#     val_loss /= len(validation_loader)
#     val_accuracy = 100. * val_correct / val_total

    print(f'     Train Loss: {train_loss:.4f}')
#     print(f'Validation Loss: {val_loss:.4f}')
    print(f'      train_acc: {train_accuracy:.2f}%\n')
#     print(f'        val_acc: {val_accuracy:.2f}%\n')

    scheduler.step()

        Epoch [1/50]
     Train Loss: 1.8357
      train_acc: 34.36%

        Epoch [2/50]
     Train Loss: 1.6495
      train_acc: 40.43%

        Epoch [3/50]
     Train Loss: 1.5646
      train_acc: 43.44%

        Epoch [4/50]
     Train Loss: 1.5216
      train_acc: 45.20%

        Epoch [5/50]
     Train Loss: 1.4842
      train_acc: 46.71%

        Epoch [6/50]
     Train Loss: 1.4556
      train_acc: 47.70%

        Epoch [7/50]
     Train Loss: 1.4297
      train_acc: 48.48%

        Epoch [8/50]
     Train Loss: 1.4048
      train_acc: 49.50%

        Epoch [9/50]
     Train Loss: 1.3810
      train_acc: 50.31%

        Epoch [10/50]
     Train Loss: 1.3606
      train_acc: 50.90%

        Epoch [11/50]
     Train Loss: 1.3425
      train_acc: 52.07%

        Epoch [12/50]
     Train Loss: 1.3222
      train_acc: 52.71%

        Epoch [13/50]
     Train Loss: 1.3051
      train_acc: 53.38%

        Epoch [14/50]
     Train Loss: 1.2883
      train_acc: 54.02%

        Epoch [

In [10]:
# 학습된 모델 평가
test_dataset = datasets.CIFAR10(root='.', train=False, download=True,
                                transform=transforms.ToTensor())
batchsize = 64
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batchsize, shuffle=True)

num_correct = 0

model.eval()

with torch.no_grad():
  for image, label in test_loader:
    image = image.cuda()
    label = label.cuda()
    output = model(image)
    pred = output.argmax(dim=1)
    num_correct += (pred == label).sum()

print(f'Accuracy : {num_correct / len(test_dataset) * 100:.2f} %')

Files already downloaded and verified
Accuracy : 63.98 %
